# Convert "Hey Nap" ONNX to TFLite

This notebook converts your ONNX model to TFLite format.

In [ ]:
# Install dependencies
!pip install -q onnx onnx2tf tensorflow
print("✅ Dependencies installed")

In [ ]:
# Upload ONNX file
from google.colab import files
import os

print("📤 Upload your hey_nap.onnx file...")
uploaded = files.upload()
onnx_file = list(uploaded.keys())[0]

# Rename to remove spaces/special chars
new_name = "hey_nap.onnx"
if onnx_file != new_name:
    os.rename(onnx_file, new_name)
    onnx_file = new_name
    print(f"✅ Renamed to: {new_name}")
else:
    print(f"✅ File: {onnx_file}")

print(f"   Size: {os.path.getsize(onnx_file) / 1024:.1f} KB")

In [ ]:
# Convert to TFLite
from pathlib import Path
import os

model_name = "hey_nap"
tflite_file = f"{model_name}.tflite"

print(f"🔄 Converting {onnx_file} to TFLite...")
print("\nEnsuring dependencies...")
!pip install -q onnx onnx2tf

print("\nConverting (may take a minute)...")
!onnx2tf -i {onnx_file} -o . -osd

# Check output
output_file = f"{model_name}_float32.tflite"
if os.path.exists(output_file):
    os.rename(output_file, tflite_file)
    print(f"\n✅ Conversion successful!")
    print(f"   File: {tflite_file}")
    print(f"   Size: {os.path.getsize(tflite_file) / 1024:.1f} KB")
else:
    print("\n⚠️  onnx2tf output not found")
    print("Trying alternative method...")

In [ ]:
# Alternative conversion (if onnx2tf failed)
print("Trying alternative method...")
!pip install -q onnx-tf

try:
    import onnx
    import tensorflow as tf
    from onnx_tf.backend import prepare
    import tempfile
    
    print("Loading ONNX...")
    onnx_model = onnx.load(onnx_file)
    
    print("Converting to TensorFlow...")
    tf_rep = prepare(onnx_model)
    
    print("Exporting...")
    with tempfile.TemporaryDirectory() as tmp_dir:
        tf_path = os.path.join(tmp_dir, "saved_model")
        tf_rep.export_graph(tf_path)
        
        print("Converting to TFLite...")
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        tflite_model = converter.convert()
        
        tflite_file = "hey_nap.tflite"
        with open(tflite_file, 'wb') as f:
            f.write(tflite_model)
        
        print(f"\n✅ Conversion successful!")
        print(f"   File: {tflite_file}")
        print(f"   Size: {os.path.getsize(tflite_file) / 1024:.1f} KB")
        
except Exception as e:
    print(f"\n❌ Failed: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Download TFLite file
from google.colab import files
import os
import glob

tflite_file = "hey_nap.tflite"

if os.path.exists(tflite_file):
    print(f"📥 Downloading {tflite_file}...")
    files.download(tflite_file)
    print(f"\n✅ Download complete!")
    print(f"\n🎉 Your TFLite model is ready!")
else:
    tflite_files = glob.glob("*.tflite")
    if tflite_files:
        print(f"Found: {tflite_files[0]}")
        files.download(tflite_files[0])
    else:
        print("❌ No TFLite file found")
        print("\nAvailable files:")
        !ls -la